## Putting `hera_sim` data into a `UVData` object

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import cmocean
import copy
from scipy import constants
from pyuvdata import UVData
from pyuvdata import utils as uvutils
from hera_sim import foregrounds, noise, rfi, sigchain

In [ ]:
def make_amp_phase_plot(data, fig, ax, log_amp=True, amp_vmin=0.5, amp_vmax=3):
    if log_amp:
        cax0 = ax[0].imshow(np.log10(np.abs(data)), vmin=amp_vmin, vmax=amp_vmax, aspect='auto')
        title = 'log(Amplitude)'
    else:
        cax0 = ax[0].imshow(np.abs(data), vmin=amp_vmin, vmax=amp_vmax, aspect='auto')
        title = 'Amplitude'
    cax1 = ax[1].imshow(np.angle(data), cmap='cmo.phase', vmin=-np.pi, vmax=np.pi, aspect='auto')
    ax[0].set_title(title)
    ax[1].set_title('Phase')
    fig.colorbar(cax0, ax=ax[0])
    fig.colorbar(cax1, ax=ax[1])
    
def bl_len_in_ns(uvw_arr):
    bl_len_ns = (np.linalg.norm(uvw_arr) / constants.c) * 1e9
    return bl_len_ns

In [ ]:
# Paths
JD_dec = '40141'
base_name = 'zen.2458106.' + JD_dec + '.xx.HH.uv'
data_in = '/data6/HERA/data/IDR2.1/2458106/' + base_name
sim_out = '/home/lwhitler/data/dfiles/sim/' + base_name.replace('.uv', '') + '.sim.uv'

In [ ]:
# Data
uvd_load = UVData()
uvd_load.read(data_in, file_type='miriad')

In [ ]:
uvd = copy.deepcopy(uvd_load)
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
cax = ax.imshow(np.abs(uvd.data_array[:, 0, :, 0]), aspect='auto', vmin=0, vmax=0.15)
fig.colorbar(cax, ax=ax)
plt.tight_layout()

In [ ]:
# Existing frequencies and LSTs from the UVData object
freqs = np.unique(uvd.freq_array) / 1e9  # In GHz for hera_sim
lsts = np.unique(uvd.lst_array)

#### Antenna-dependent things (antenna gains)

In [ ]:
ants = list(set(uvd.ant_1_array).union(set(uvd.ant_2_array)))

In [ ]:
# Antenna gains
gains = sigchain.gen_gains(freqs, ants)

#### Non-baseline dependent things (RFI and noise)

In [ ]:
Tsky_model = noise.HERA_Tsky_mdl['xx']

# Noise
Tsky = noise.resample_Tsky(freqs, lsts, Tsky_mdl=Tsky_model)
Trx = 150.  # ...Receiver temperature?
noise_jy = noise.sky_noise_jy(Tsky + Trx, freqs, lsts)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 5))
make_amp_phase_plot(noise_jy, fig, ax, amp_vmin=0.5, amp_vmax=2)
plt.tight_layout()

In [ ]:
# RFI
rfi_narrow = rfi.rfi_stations(freqs, lsts)
rfi_broad = rfi.rfi_impulse(freqs, lsts, chance=0.025)
rfi_scatter = rfi.rfi_scatter(freqs, lsts, chance=0.001)
rfi_all = rfi_narrow + rfi_broad + rfi_scatter

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 5))
make_amp_phase_plot(rfi_all, fig, ax)
plt.tight_layout()

In [ ]:
# Noise and RFI
noise_rfi = rfi_all + noise_jy

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 5))
make_amp_phase_plot(noise_rfi, fig, ax, amp_vmin=0.5, amp_vmax=2)
plt.tight_layout()

#### Baseline-dependent things (foregrounds)

In [ ]:
unique_bls, bl_inds = np.unique(uvd.baseline_array, return_index=True)
bl_len_ns = bl_len_in_ns(uvd.uvw_array[bl_inds])

In [ ]:
# Find redundant baseline groups from unique baselines
reds, bin_ctrs, lens, conj_bl_nums = uvutils.get_baseline_redundancies(unique_bls, uvd.uvw_array[bl_inds],
                                                                       tol=0.5, with_conjugates=True)
# Throw out autos
auto_ind = np.where(lens == 0)
del reds[auto_ind[0][0]]
bin_ctrs = np.delete(bin_ctrs, auto_ind)
lens = np.delete(lens, auto_ind)

In [ ]:
# Baseline lengths in ns for each redundant group
bl_lens_ns = [bl_len_in_ns(length) for length in lens]

In [ ]:
# Start with diffuse foregrounds for each redundant group
fgs = {}
for i, len_ns in enumerate(bl_lens_ns):
    diffuse = foregrounds.diffuse_foreground(Tsky_model, lsts, freqs, len_ns)
    pt_src = foregrounds.pntsrc_foreground(lsts, freqs, len_ns, nsrcs=200)
    fgs[i] = diffuse + pt_src

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(7, 5))
make_amp_phase_plot(fgs[0], fig, ax)
plt.tight_layout()

#### Apply gains to simulated data and save it

The autos seem to make `hera_sim` unhappy, so I set them to NaNs for now... there's hopefully a better solution, though.

In [ ]:
true_vis = {}
for key in fgs.keys():
    true_vis[key] = fgs[key] + noise_rfi

In [ ]:
bl_dict = dict.fromkeys(np.unique(uvd.baseline_array))
for red_group in reds:
    red_ind = reds.index(red_group)
    true_vis_red = true_vis[red_ind]
    for bl_num in red_group:
        if bl_num in conj_bl_nums:
            true_vis_red = true_vis_red.conj()
        bl_tuple = uvd.baseline_to_antnums(bl_num)
        g_ij = gains[bl_tuple[0]] * gains[bl_tuple[1]].conj()
        bl_vis = true_vis_red * g_ij
        if bl_num in bl_dict.keys():
            bl_dict[bl_num] = bl_vis
        else:
            print('Baseline number {0} is not in the baseline dictionary.'.format(bl_num))
for bl_num in bl_dict.keys():
    if bl_dict[bl_num] is None:
        bl_dict[bl_num] = np.full((len(lsts), len(freqs)), np.nan + 1j*np.nan)
        print('Setting baseline {0} to NaNs.'.format(uvd.baseline_to_antnums(bl_num)))

In [ ]:
sim_data = np.zeros_like(uvd.data_array)
for bl_num in bl_dict.keys():
    ind_blt = np.where(uvd.baseline_array == bl_num)[0]
    sim_data[ind_blt] = bl_dict[bl_num][:, None, :, None]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
cax = ax.imshow(np.abs(sim_data[:, 0, :, 0]), aspect='auto', vmin=0, vmax=0.15)
fig.colorbar(cax, ax=ax)
plt.tight_layout()

#### Save the simulated data to a miriad file

In [ ]:
uvd_sim = copy.deepcopy(uvd_load)
# Unflag everything just in case anything was flagged
uvd_sim.flag_array = np.full_like(uvd.flag_array, False)
uvd_sim.data_array = sim_data
uvd_sim.write_miriad(sim_out)